# Long Beach Gas & Oil - Consumption Monthly Average
This kernel looks into the three-year (2014 - 2016)data containing the gas and oil consumption in Long Beach, CA.
Datasets are from the Long Beach City DataLB website: https://prod-longbeachca.hub.arcgis.com/search

In [1]:
import pandas as pd
import altair as alt
import numpy as np

## Data Import and Preprocessing

In [2]:
df_2014 = pd.read_csv('LB_Avg_Consumption_Gas_Oil_2014.csv')
df_2015 = pd.read_csv('LB_Avg_Consumption_Gas_Oil_2015.csv')
df_2016 = pd.read_csv('LB_Avg_Consumption_Gas_Oil_2016.csv')

#### Important steps in preprocesing of data include:
* Dropping the following columns as they are not needed in analysis: `OBJECT_ID`, `LAST_UPD`, `OBJECTID`
* Concatenating the three dataframes: `df_2014`, `df_2015`, `df_2016`
* Renaming the columns
* Parsing `Date` column

In [3]:
# Dropping unwanted columns/features
df_2014 = df_2014.drop(['OBJECT_ID','LAST_UPD','OBJECTID'], axis=1)
df_2015 = df_2015.drop(['OBJECT_ID','LAST_UPD','OBJECTID'], axis=1)
df_2016 = df_2016.drop(['OBJECT_ID','LAST_UPD','OBJECTID'], axis=1)

# Dataframe concatenation

df = pd.concat([df_2014, df_2015, df_2016])
df.reset_index(drop=True)

# Renaming and parsing

df.columns = ['Date','Customer','Consumption']
df['Date'] = pd.to_datetime(df['Date'])
df.sort_values(by=['Date'], inplace=True, ascending=True)

### Data Statistics
* The concatenated dataframe has 315 rows and 3 main columns.
* There are no missing count.

In [4]:
df.shape

(315, 3)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 315 entries, 0 to 9
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Date         315 non-null    datetime64[ns]
 1   Customer     315 non-null    object        
 2   Consumption  315 non-null    float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 9.8+ KB


In [6]:
df.isna().sum()

Date           0
Customer       0
Consumption    0
dtype: int64

## Data Visualization

### Customer Type
* Top four customers with the highest gas consumption were:
    - Gas - CNG (Compressed Natural Gas)
    - Gas - Large Commercial & Industrial
    - Gas - Electric Generation
    - Gas - Transportation & Exchange
* Residential customers had stable yearly consumption during the three-year period.

In [7]:
# Gas & Oil - Monthly Consumption Average (2014 - 2016)

highlight = alt.selection_multi(on='mouseover')

a0 = alt.Chart(df).mark_line().encode(
    x= alt.X('Date:T'),
    y='Consumption:Q',
    color= alt.condition(highlight,'Customer', alt.value('lightgray')),
    size=alt.condition(~highlight, alt.value(1), alt.value(3))
).properties(
    width=600, height=300,
    title='Monthly Consumption Average (2014 - 2016)',
    selection=highlight
)

# Plotting the top four consumers

highlight = alt.selection_multi(on='mouseover')

a1 = alt.Chart(df).mark_line().encode(
    x = alt.X('Date:T'),
    y = 'Consumption:Q',
    color= alt.condition(highlight,'Customer', alt.value('lightgray')),
    size=alt.condition(~highlight, alt.value(1), alt.value(3))
).properties(
    width=600, height=300,
    selection=highlight,
    title='Top four Customers with the Highest Gas/Oil Consumption'
).transform_filter(
    alt.FieldOneOfPredicate(field='Customer', oneOf=['Gas - CNG (Compressed Natural Gas)',
                                                     'Gas - Large Commercial & Industrial',
                                                     'Gas - Electric Generation',                                                    
                                                     'Gas - Transportation & Exchange']
                                                     )
)

a2 = alt.Chart(df).mark_line().encode(
    x = alt.X('Date:T'),
    y = 'Consumption:Q',
    color= alt.condition(highlight,'Customer', alt.value('lightgray')),
    size=alt.condition(~highlight, alt.value(1), alt.value(3))
).properties(
    width=600, height=300,
    selection=highlight,
    title='Consumption of Residential Customers'
).transform_filter(
    alt.FieldOneOfPredicate(field='Customer', oneOf=['Gas - Residential - Single Family',
                                                     'Gas - Residential - Multi-Family',
                                                     'Gas - Residential - Duplex']
                                                     )
)

(a0 & a1 & a2).resolve_scale(color='independent')

alt.VConcatChart(...)

### Interactive Graph
Below are the yearly and monthly trends depicting the consumption of gas and oil by each customer.
* There was noticeable increase in gas consumption during the fall/winter months (October - December) as most customers use their heating systems.
* A downward trend can be seen towards the end of winter, to spring and summer months as heating systems were less used during this time of the year.
* From 2014 to 2016, the yearly consumption of residential properties were stable.
* During the three-year period, there was significant drop in the consumption of gas by Large Commercial and Industrial customers.
* Increasing consumption trend by CNG and Electric Generation customers every year.

In [8]:
cust_dr = alt.binding_select(options=['Gas - CNG (Compressed Natural Gas)',
                                      'Gas - Commercial & Industrial',
                                      'Gas - Transportation & Exchange',
                                      'Gas - Residential - Single Family',
                                      'Gas - Residential - Duplex',
                                      'Gas - Residential - Multi-Family',
                                      'Gas - Electric Generation',
                                      'Gas - Large Commercial & Industrial',
                                      'Gas - Small Commercial & Industrial'])
selection = alt.selection_single(fields=['Customer'], bind=cust_dr, name='Select', clear='click')

d1 = alt.Chart(df).mark_bar().encode(
    x = alt.X('Date:O', timeUnit='year', title='Year'),
    y = alt.Y('mean(Consumption):Q', title='Average Consumption'),
    color=alt.Color('Date:O', timeUnit='year')
).properties(
    width=300, height=300,
    title='Yearly Average Consumption',
    selection = selection
).transform_filter(selection)

d2 = alt.Chart(df).mark_line(strokeWidth=2).encode(
    x = alt.X('Date:O', timeUnit='month', title='Month'),
    y = 'Consumption:Q',
    color=alt.Color('Date:O', timeUnit='year', title='Year', scale=alt.Scale(scheme='tableau10')),
).properties(
    width=300, height=300,
    title='Consumption Monthly Trend',
    selection = selection
).transform_filter(selection)

d = (d1 | d2).properties(title='Gas and Oil Consumption')
d.configure_title(color='darkblue',
                  anchor='middle')

alt.HConcatChart(...)